# <span style='background-color: #fff5b1'>오늘의 한 걸음  🚶🏽‍♂️: 경구약제 이미지 분류</span>

## Contexts

### 1. READY
    1-1 오늘의 Exp와 Rubric  
    1-2 사용하는 라이브러리  

### 2. GAME
    2-1. 데이터 읽어오기  
    2-2. 데이터 전처리  
    2-3. 모델 학습  
    2-4. 데이터 평가   

### 3. POTG
    3-1. 소감
    3-2. 어려웠던 점과 극복방안  
    3-3. 추후  

---


# 1. Ready
## 1-1. 오늘의 Exp와 Rubric

우리나라에서는 AI Hub 라는 웹페이지에서  
다양한 이미지를 제공하고 있다. 

이번에 우리가 다룰 데이터는 경구약제 이미지 데이터이다.
경구약 (삼켜서 먹는 알약) 제품 약 5000종의 사진이  
한 종류당 약 1250장.

도합 2백만 장의 약 사진 및 메타 데이터를 가지고 있다

자료 : <a href='https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&aihubDataSe=realm&dataSetSn=576'>AI_Hub : 경구약제 이미지 데이터</a>

시각장애인 혹은 사회적 약자들은 약을 섭취함에 있어서  
섭취를 혼동하거나 남용하는 문제와 직면한다.

이번 Exp 는 경구약제 약품을 분류하고 식별 결과를 안내할 수 있는 AI 분류기를 만드는 것이다!

우선 기술상의 문제로
1TB가 넘는 용량의 사진을 모두 이용하기에 부담이 커서  

Traininig data set 에 있는 1250장의 사진 데이터를 이용하기로 했다.

## 1-2. 사용하는 라이브러리

In [32]:
import os
import json
import glob
import pandas as pd


# 2. GAME
## 2-1. 데이터 읽어오기

일단 어떤 meta data 가 있는지 직접 눈으로 확인해보았다.

In [5]:
combined = []


for json_file in glob.glob("./pills/pills_data/data/Training/label/pills_5000/TL_81/K-038884_json/K-038884_0_0_0_0_75_000_200.json") : 

    with open(json_file, "rb") as infile:

        combined.append(json.load(infile))


In [8]:
type(json_file)

str

In [6]:
combined

[{'images': [{'file_name': 'K-038884_0_0_0_0_75_000_200.png',
    'width': 976,
    'height': 1280,
    'imgfile': 'K-038884_0_0_0_0_75_000_200.png',
    'drug_N': 'K-038884',
    'drug_S': '정상알약',
    'back_color': '검은색 배경',
    'drug_dir': '앞면',
    'light_color': '전구색',
    'camera_la': 75,
    'camera_lo': 0,
    'size': 200,
    'item_seq': '201602513',
    'leng_long': '6.2',
    'leng_short': '6.2',
    'thick': '3.05',
    'id': 1,
    'dl_idx': '38883',
    'dl_mapping_code': 'K-038884',
    'dl_name': '에피나레정',
    'dl_name_en': 'Epinare Tab.',
    'img_key': 'http://connectdi.com/design/img/drug/1NOwp2F69E9.jpg',
    'dl_material': '에피나스틴염산염',
    'dl_material_en': 'Epinastine Hydrochloride',
    'dl_custom_shape': '정제, 저작정',
    'dl_company': '제이더블유신약(주)',
    'dl_company_en': 'Jw Shinyak',
    'di_company_mf': '삼천당제약(주)',
    'di_company_mf_en': 'Scd Pharm',
    'di_item_permit_date': '20160504',
    'di_class_no': '[01490]기타의 알레르기용약',
    'di_etc_otc_code': '전문의약품',
    'd

약에 대해 알려주는 많은 데이터가 있었지만,  
우리의 경우 이미지 분류를 할 것이기 떄문에  
연동된 이미지 경로와 이름만을 뽑아내려고 계획했다.

따라서 json 내부에 두 데이터를 뽑아 csv 파일로 만들어 저장했다.

In [69]:
PATH = './pills/pills_data/data/Training/label/pills_5000/TL_81/'

def save_csv(PATH, name):
    """
    지정한 경로의 경구약제 json 파일을
    name 이름으로 csv 저장

    Args:
        PATH (_str_): 초기 경로
        name (_str_): 이름
    """
    files = os.listdir(PATH)

    cnt = 0
    pills_df = pd.DataFrame({'file':[],
                                'dl_name':[]}) # DataFrame
    for file_name in files:
        json_path = os.listdir(PATH + file_name)

        for json_data in json_path:
            json_file =  glob.glob(PATH + file_name +'/'+ json_data )[0]
            with open(json_file, "rb") as infile:
                data = json.load(infile)
                file = data['images'][0]['file_name']
                dl_name = data['images'][0]['dl_name']
                pills_df = pills_df.append({'file':[file], 'dl_name':[dl_name]}, ignore_index=True)
    
    pills_df.to_csv(name)
    return

save_csv(PATH, 'label.csv')


In [74]:
df = pd.read_csv('label.csv')
df

,Unnamed: 0,file,name,dl_name
0,0,['K-038884_0_0_0_0_75_000_200.png'],NaN,['에피나레정']
1,1,['K-038884_0_0_0_0_75_020_200.png'],NaN,['에피나레정']
2,2,['K-038884_0_0_0_0_75_040_200.png'],NaN,['에피나레정']
3,3,['K-038884_0_0_0_0_75_060_200.png'],NaN,['에피나레정']
4,4,['K-038884_0_0_0_0_75_080_200.png'],NaN,['에피나레정']
...,...,...,...,...
9607,9607,['K-039147_0_2_1_2_90_260_200.png'],NaN,['듀카브정30/10밀리그램']
9608,9608,['K-039147_0_2_1_2_90_280_200.png'],NaN,['듀카브정30/10밀리그램']
9609,9609,['K-039147_0_2_1_2_90_300_200.png'],NaN,['듀카브정30/10밀리그램']
9610,9610,['K-039147_0_2_1_2_90_320_200.png'],NaN,['듀카브정30/10밀리그램']


In [ ]:
import pandas as df
df.drop('variety', axis=1)

## 2-2. 데이터 전처리

## 2-3. 모델 학습

## 2-4. 데이터 평가

# 3. POTG
## 3-1. 소감

## 3-2. 어려웠던 점과 극복방안

## 3-3. 추후